In [2]:
import os

In [3]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction/research'

In [4]:
os.chdir('../')

In [99]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction'

In [100]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tejas05in"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "9efcb5c7b79d0e949378459b922b1462a80fa413"

In [101]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [102]:
from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml , create_directories

In [103]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """
        This function returns the ModelEvaluationConfig object that contains all the configuration parameters required for model evaluation.

        Args:
            None

        Returns:
            ModelEvaluationConfig: The ModelEvaluationConfig object containing all the configuration parameters

        """
        config = self.config.model_evaluation
        params = self.params.GradientBoostedTreesLearner
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
        )

        return model_evaluation_config

In [104]:
import os
import pandas as pd
from urllib.parse import urlparse
import mlflow
import numpy as np
import ydf
from mlflow.data.pandas_dataset import PandasDataset

In [105]:
class CustomModelWrapper(mlflow.pyfunc.PythonModel):
   def load_context(self, context):
       import ydf
       self.model = ydf.load_model(
           context.artifacts["model"])
       return self.model

   def predict(self, model_input):
       return self.model.predict(model_input)





class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    

    def log_into_mlflow(self):
        """
        This function logs the metrics and model into mlflow.

        Args:
            None

        Returns:
            None

        """

        # Load the test data
        test_data = pd.read_csv(self.config.test_data_path)

        # Load the model
        model = ydf.load_model(self.config.model_path)

        # Evaluate the model on the test data
        evaluation = model.evaluate(test_data)
        

        with open(self.config.metric_file_name, "w") as f:
            # storing the training description in a html file
            f.write(evaluation.html())
        
        # print(evaluation.to_dict())
        # print(evaluation.confusion_matrix)
        # Set the mlflow tracking uri
        mlflow.set_registry_uri(self.config.mlflow_uri)

        # Get the type of the tracking uri
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        dataset: PandasDataset = mlflow.data.from_pandas(
            test_data)
    
        # # Start a mlflow run
        with mlflow.start_run():
            
            #log data
            mlflow.log_input(dataset, context="testing")

            # Log the parameters
            mlflow.log_params(self.config.all_params)

            # Log the metrics
            mlflow.log_metric("loss", evaluation.loss)
            mlflow.log_metric("accuray", evaluation.accuracy)
            
            # Register the model
            # There are other ways to use the model registry , which depends on the use case
            # Please refer to the documentation for more information
            # https://mlflow.org/docs/latest/model-registry.html #api-workflow
            if tracking_uri_type_store != 'file':
                mlflow.pyfunc.log_model(
                    "model", python_model=CustomModelWrapper(), artifacts={"model": self.config.model_path}, registered_model_name="GradientBoostedTreesModel", pip_requirements='requirements.txt')
            else:
                mlflow.pyfunc.log_model("model", python_model=CustomModelWrapper(), artifacts={
                                        "model": self.config.model_path}, registered_model_name="GradientBoostedTreesModel")

In [106]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-02-03 21:06:11,629:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-02-03 21:06:11,631:INFO:common:yaml file: params.yaml loaded successfully]
[2024-02-03 21:06:11,634:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-02-03 21:06:11,634:INFO:common:created directory at :artifacts]
[2024-02-03 21:06:11,635:INFO:common:created directory at :artifacts/model_evaluation]
[2024-02-03 21:06:11,641:INFO:model_lib:Loading model from artifacts/model_trainer/model]


/home/tejas/MLProj/Thyroid-Disease-Prediction/env/lib/python3.11/site-packages/mlflow/data/digest_utils.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]


[2024-02-03 21:08:11,796:WARNING:connectionpool:Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/create]


/home/tejas/MLProj/Thyroid-Disease-Prediction/env/lib/python3.11/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/home/tejas/MLProj/Thyroid-Disease-Prediction/env/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the 